In [37]:
#Dependencies
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
import sqlite3

# EXTRACT DATA

In [38]:
#Read data from science journal
#assets/data/
scienceJournal_plasticData = pd.read_csv("1260352_SupportingFile_Suppl._Excel_seq1_v2.csv")
country_geocode=pd.read_csv("coutry_geocodes.csv")

Extract EPA data

In [39]:
# read files and put in separate dataframes
epa_files = ["1 Materials Generated in the U.S. Municipal Waste Stream, 1960 to 2015 (in tons).csv", 
            "2 Materials Recycled and Composted in the U.S. Municipal Waste Stream, 1960 to 2015 (in tons).csv",
           "3 Materials Combusted with Energy Recovery in the U.S. Municipal Waste Stream, 1960 to 2015 (in tons).csv",
           "4 Materials Landfilled in the U.S. Municipal Waste Stream, 1960 to 2015 (in tons).csv"]

li = []

for filename in epa_files:
    df = pd.read_csv(filename)
    df1 = df.iloc[[0,1,5,6,9],:]
    df2 = df1.T
    df2.columns = ['paper_paperboard', 'glass', 'metals', 'plastic', 'wood']
    if filename == epa_files[0]:
        df2["type_id"] = "1"
        df2["waste_type"] = "generated"
    elif filename == epa_files[1]:
        df2["type_id"] = "2"
        df2["waste_type"] = "recycled"
    elif filename == epa_files[2]:
        df2["type_id"] = "3"
        df2["waste_type"] = "combusted"
    else:
        df2["type_id"] = "4"
        df2["waste_type"] = "landfilled"
    frame = df2.iloc[1:]
    li.append(frame)

# TRANSFORM DATA

In [28]:
scienceJournal_plasticData=scienceJournal_plasticData.dropna()
global_plastic_df=scienceJournal_plasticData

#Rename columns
global_plastic_df=global_plastic_df.rename(columns = {"Waste_generation_kg/day": "All_Waste_Generated_kg/day",
                                                     "%_Plastic_in_waste_stream":"Per_Plastic_in_waste_stream",
                                                     "%_Inadequately_managed_waste": "Per_Inadequately_managed_waste",
                                                     "%_Littered_waste": "Per_Littered_waste"})

global_plastic_df.head()

,Country,Economic_status,Coastal_population,Waste_generation_rate_kg/person/day_All_Waste,Per_Plastic_in_waste_stream,Per_Inadequately_managed_waste,Per_Littered_waste,All_Waste_Generated_kg/day,Plastic_waste_generation_kg/day,Inadequately_managed_plastic_waste_kg/day,Plastic_waste_littered_[kg/day],Mismanaged_plastic_waste_kg/person/day,Mismanaged_plastic_waste_in_2010_tonnes,Mismanaged_plastic_waste_in_2025_tonnes
0,Albania,LMI,2530533.0,0.77,9.0,45.0,2.0,1948510.0,174392.0,77897.0,3488.0,0.03,29705.0,63051.0
1,Algeria,UMI,16556580.0,1.20,12.0,58.0,2.0,19867896.0,2374214.0,1378693.0,47484.0,0.09,520555.0,1017444.0
2,Angola,LMI,3790041.0,0.48,13.0,71.0,2.0,1819220.0,235589.0,166597.0,4712.0,0.05,62528.0,136770.0
3,Anguilla,HIC,14561.0,2.10,12.0,2.0,2.0,30578.0,3654.0,68.0,73.0,0.01,52.0,73.0
4,Antigua & Barbuda,HIC,66843.0,5.50,12.0,6.0,2.0,367637.0,43933.0,2555.0,879.0,0.05,1253.0,1385.0


In [29]:
#Needed to check to see if the values are integers
global_plastic_df["Mismanaged_plastic_waste_in_2010_tonnes"].head()

0     29705.0
1    520555.0
2     62528.0
3        52.0
4      1253.0
Name: Mismanaged_plastic_waste_in_2010_tonnes, dtype: float64

In [30]:
#Add % of change from 2010 to 2025 for Mismanaged Waste
global_plastic_df["Per_Change_2010_to_2025"]=(global_plastic_df["Mismanaged_plastic_waste_in_2025_tonnes"]-global_plastic_df["Mismanaged_plastic_waste_in_2010_tonnes"])/global_plastic_df["Mismanaged_plastic_waste_in_2010_tonnes"]
global_plastic_df.head()

,Country,Economic_status,Coastal_population,Waste_generation_rate_kg/person/day_All_Waste,Per_Plastic_in_waste_stream,Per_Inadequately_managed_waste,Per_Littered_waste,All_Waste_Generated_kg/day,Plastic_waste_generation_kg/day,Inadequately_managed_plastic_waste_kg/day,Plastic_waste_littered_[kg/day],Mismanaged_plastic_waste_kg/person/day,Mismanaged_plastic_waste_in_2010_tonnes,Mismanaged_plastic_waste_in_2025_tonnes,Per_Change_2010_to_2025
0,Albania,LMI,2530533.0,0.77,9.0,45.0,2.0,1948510.0,174392.0,77897.0,3488.0,0.03,29705.0,63051.0,1.122572
1,Algeria,UMI,16556580.0,1.20,12.0,58.0,2.0,19867896.0,2374214.0,1378693.0,47484.0,0.09,520555.0,1017444.0,0.954537
2,Angola,LMI,3790041.0,0.48,13.0,71.0,2.0,1819220.0,235589.0,166597.0,4712.0,0.05,62528.0,136770.0,1.187340
3,Anguilla,HIC,14561.0,2.10,12.0,2.0,2.0,30578.0,3654.0,68.0,73.0,0.01,52.0,73.0,0.403846
4,Antigua & Barbuda,HIC,66843.0,5.50,12.0,6.0,2.0,367637.0,43933.0,2555.0,879.0,0.05,1253.0,1385.0,0.105347


In [31]:
#First Schema - Country Information 
country_waste=global_plastic_df[["Country","Economic_status", "Coastal_population", 
                                 "All_Waste_Generated_kg/day",
                                 "Per_Plastic_in_waste_stream"]]

#country_waste=country_waste.sort_values(by=["Waste_generation_kg/day"])
#country_waste=country_waste.groupby(["Economic_status"]).mean()
country_waste.rename(columns = {"All_Waste_Generated_kg/day": "All_Waste_Generated_kg_day"}).head()

,Country,Economic_status,Coastal_population,All_Waste_Generated_kg_day,Per_Plastic_in_waste_stream
0,Albania,LMI,2530533.0,1948510.0,9.0
1,Algeria,UMI,16556580.0,19867896.0,12.0
2,Angola,LMI,3790041.0,1819220.0,13.0
3,Anguilla,HIC,14561.0,30578.0,12.0
4,Antigua & Barbuda,HIC,66843.0,367637.0,12.0


In [32]:
#Second Schema - Country Plastic Waste
country_plastic_waste=global_plastic_df[["Country","Plastic_waste_generation_kg/day"]]

managed_plastic_waste=global_plastic_df["Plastic_waste_generation_kg/day"]-global_plastic_df["Inadequately_managed_plastic_waste_kg/day"]-global_plastic_df["Plastic_waste_littered_[kg/day]"]

mismanaged_plastic_waste=global_plastic_df["Inadequately_managed_plastic_waste_kg/day"]+global_plastic_df["Plastic_waste_littered_[kg/day]"]


country_plastic_waste["Managed_plastic_waste_kg/day"]=managed_plastic_waste
country_plastic_waste["Mismanaged_plastic_waste_kg/day"]=mismanaged_plastic_waste
country_plastic_waste.rename(columns = {"Plastic_waste_generation_kg/day": "Plastic_waste_generation_kg_day",
                                       "Managed_plastic_waste_kg/day": "Managed_plastic_waste_kg_day",
                                        "Mismanaged_plastic_waste_kg/day":"Mismanaged_plastic_waste_kg_day"}).head()

C:\Users\aruba\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\aruba\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,Country,Plastic_waste_generation_kg_day,Managed_plastic_waste_kg_day,Mismanaged_plastic_waste_kg_day
0,Albania,174392.0,93007.0,81385.0
1,Algeria,2374214.0,948037.0,1426177.0
2,Angola,235589.0,64280.0,171309.0
3,Anguilla,3654.0,3513.0,141.0
4,Antigua & Barbuda,43933.0,40499.0,3434.0


In [36]:
#Third Schema Country Plastic Waste by Percent Managed & Mismanaged
percent_managed_plastic_waste=country_plastic_waste["Managed_plastic_waste_kg/day"]/country_plastic_waste["Plastic_waste_generation_kg/day"]
percent_mismanaged_plastic_waste=country_plastic_waste["Mismanaged_plastic_waste_kg/day"]/country_plastic_waste["Plastic_waste_generation_kg/day"]

country_plastic_waste_per=global_plastic_df[["Country"]]
country_plastic_waste_per["Per_Managed_plastic_waste_kg/day"]=percent_managed_plastic_waste
country_plastic_waste_per["Per_Mismanaged_plastic_waste_kg/day"]=percent_mismanaged_plastic_waste
country_plastic_waste_per.rename(columns = {"Per_Managed_plastic_waste_kg/day": "Per_Managed_plastic_waste_kg_day",
                                       "Per_Mismanaged_plastic_waste_kg/day": "Pec_Mismanaged_plastic_waste_kg_day"}).head()

C:\Users\aruba\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\aruba\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Country,Per_Managed_plastic_waste_kg_day,Pec_Mismanaged_plastic_waste_kg_day
0,Albania,0.533321,0.466679
1,Algeria,0.399306,0.600694
2,Angola,0.272848,0.727152
3,Anguilla,0.961412,0.038588
4,Antigua & Barbuda,0.921836,0.078164


In [40]:
#Fourth Schema per person
pperson_plastic_waste=global_plastic_df[["Country","Waste_generation_rate_kg/person/day_All_Waste",
                                        "Mismanaged_plastic_waste_kg/person/day"]]
pperson_plastic_waste.rename(columns = {"Waste_generation_rate_kg/person/day_All_Waste": "Waste_generation_rate_kg_person_day_All_Waste",
                                       "Mismanaged_plastic_waste_kg/person/day": "Mismanaged_plastic_waste_kg_person_day"}).head()

,Country,Waste_generation_rate_kg_person_day_All_Waste,Mismanaged_plastic_waste_kg_person_day
0,Albania,0.77,0.03
1,Algeria,1.20,0.09
2,Angola,0.48,0.05
3,Anguilla,2.10,0.01
4,Antigua & Barbuda,5.50,0.05


In [44]:
#Fith Schema Mismanaged 2010 and 2025 and difference
mismanaged_2010_to_2025=global_plastic_df[["Country","Mismanaged_plastic_waste_in_2010_tonnes",
                                                                 "Mismanaged_plastic_waste_in_2025_tonnes", "Per_Change_2010_to_2025"]]

mismanaged_2010_to_2025.head()

,Country,Mismanaged_plastic_waste_in_2010_tonnes,Mismanaged_plastic_waste_in_2025_tonnes,Per_Change_2010_to_2025
0,Albania,29705.0,63051.0,1.122572
1,Algeria,520555.0,1017444.0,0.954537
2,Angola,62528.0,136770.0,1.187340
3,Anguilla,52.0,73.0,0.403846
4,Antigua & Barbuda,1253.0,1385.0,0.105347


In [45]:
#6th Schema per person impact
#1 kg = 2.20462
#plastic to ocean = .25
mismanaged_plastic_pperson=global_plastic_df[["Country","Mismanaged_plastic_waste_kg/person/day"]]
mismanaged_plastic_pperson=pd.DataFrame(mismanaged_plastic_pperson)

mismanaged_plastic_pperson_lb=round(mismanaged_plastic_pperson["Mismanaged_plastic_waste_kg/person/day"].astype("float")*2.20462,2)
mismanaged_plastic_pperson_lb_ocean=round(mismanaged_plastic_pperson_lb*.25,2)

mismanaged_plastic_pperson["Mismanaged_plastic_waste_lb/person/day"]=mismanaged_plastic_pperson_lb
mismanaged_plastic_pperson["Mismanaged_plastic_waste_lb/person/day_to_ocean"]=mismanaged_plastic_pperson_lb_ocean
mismanaged_plastic_pperson.rename(columns = {"Mismanaged_plastic_waste_lb/person/day":"Mismanaged_plastic_waste_lb_person_day",
                                             "Mismanaged_plastic_waste_lb/person/day_to_ocean":"Mismanaged_plastic_waste_lb_person_day_to_ocean"}).head()

,Country,Mismanaged_plastic_waste_kg/person/day,Mismanaged_plastic_waste_lb_person_day,Mismanaged_plastic_waste_lb_person_day_to_ocean
0,Albania,0.03,0.07,0.02
1,Algeria,0.09,0.20,0.05
2,Angola,0.05,0.11,0.03
3,Anguilla,0.01,0.02,0.00
4,Antigua & Barbuda,0.05,0.11,0.03


Transform EPA data

In [46]:
# individually clean df and drop nans

# generated waste
gw = li[0].iloc[:-5]
# recycled waste
rw = li[1]
# combusted waste
cw = li[2]
# landfilled
lw = li[3]

In [47]:
masterdf = pd.concat([gw,rw,cw,lw], ignore_index=False)
masterdf2 = masterdf.rename_axis('year').reset_index()
masterdf2.to_csv("masterdf2.csv")

In [48]:
# table for waste types
wastetypes = [["generated", 1], ["recycled", 2], ["combusted",3], ["landfilled",4]]
typedf = pd.DataFrame(wastetypes, columns = ["waste_type", "type_id"])
typedf.to_csv("typedf.csv")

# LOAD DATA

In [24]:
# connection build
#connection_string = "root:root@localhost/plasticdb"
#engine = create_engine(f'mysql://{connection_string}')

In [49]:
engine = create_engine("sqlite:///plastichorror.sqlite")
conn = engine.connect()

In [50]:
# load global waste data
country_waste.to_sql(name= "country_waste_info", con=engine, if_exists='append', index=False)
country_plastic_waste.to_sql(name= "country_plastic_waste", con=engine, if_exists='append', index=False)
country_plastic_waste_per.to_sql(name= "plastic_waste_percentage", con=engine, if_exists='append', index=False)
pperson_plastic_waste.to_sql(name= "pperson_plastic_waste", con=engine, if_exists='append', index=False)
mismanaged_2010_to_2025.to_sql(name= "mismanaged_pwaste_2010_to_2025", con=engine, if_exists='append', index=False)
mismanaged_plastic_pperson.to_sql(name= "mismanaged_plastic_pperson", con=engine, if_exists='append', index=False)
country_geocode.to_sql(name= "country_geocode", con=engine, if_exists='append', index=False)

In [51]:
# load epa data
masterdf.to_sql(name='master', con= engine, if_exists='append', index=False)
typedf.to_sql(name='waste_types', con= engine, if_exists='append', index=False)

In [52]:
# check table names
engine.table_names()

['country_geocode',
 'country_plastic_waste',
 'country_waste_info',
 'master',
 'mismanaged_plastic_pperson',
 'mismanaged_pwaste_2010_to_2025',
 'plastic_waste_percentage',
 'pperson_plastic_waste',
 'waste_types']